# Importing necessary libraries

In [ ]:
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from keras.utils import to_categorical
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPool2D, Activation, Dropout
from keras.constraints import maxnorm
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

# Declaring constants

In [ ]:
#Declaring constants
FIG_WIDTH=20 # Width of figure
ROW_HEIGHT=3 # Height of each row when showing a figure which consists of multiple rows
RESIZE_DIM=28# The images will be resized to 28x28 pixels
PIXEL_ADJ = 3 # for adjusting the image croping 

# Creating Path Files

In [ ]:


data_dir=os.path.join('G:/Numta_Workshop/Numta_Workshop')
paths_train_a=glob.glob(os.path.join(data_dir,'training-a','*.png'))
paths_train_b=glob.glob(os.path.join(data_dir,'training-b','*.png'))
paths_train_e=glob.glob(os.path.join(data_dir,'training-e','*.png'))
paths_train_c=glob.glob(os.path.join(data_dir,'training-c','*.png'))
paths_train_d=glob.glob(os.path.join(data_dir,'training-d','*.png'))
paths_train_all=paths_train_a+paths_train_b+paths_train_c+paths_train_d+paths_train_e

paths_test_a=glob.glob(os.path.join(data_dir,'testing-a','*.png'))
paths_test_b=glob.glob(os.path.join(data_dir,'testing-b','*.png'))
paths_test_e=glob.glob(os.path.join(data_dir,'testing-e','*.png'))
paths_test_c=glob.glob(os.path.join(data_dir,'testing-c','*.png'))
paths_test_d=glob.glob(os.path.join(data_dir,'testing-d','*.png'))
paths_test_f=glob.glob(os.path.join(data_dir,'testing-f','*.png'))+glob.glob(os.path.join(data_dir,'testing-f','*.JPG'))
paths_test_auga=glob.glob(os.path.join(data_dir,'testing-auga','*.png'))
paths_test_augc=glob.glob(os.path.join(data_dir,'testing-augc','*.png'))
paths_test_all=paths_test_a+paths_test_b+paths_test_c+paths_test_d+paths_test_e+paths_test_f+paths_test_auga+paths_test_augc

path_label_train_a=os.path.join(data_dir,'training-a.csv')
path_label_train_b=os.path.join(data_dir,'training-b.csv')
path_label_train_e=os.path.join(data_dir,'training-e.csv')
path_label_train_c=os.path.join(data_dir,'training-c.csv')
path_label_train_d=os.path.join(data_dir,'training-d.csv')

# Utility Functions and Functions that make the dataset augmented

In [ ]:
def get_key(path):
    # seperates the key of an image from the filepath
    key=path.split(os.sep)[-1] # os.sep = '/' and [-1] refers to the last element of the list
    return key

#Data-set doesn't contain horizontal or vertical flips, worthy enough to be ignored.

#Random-rotating the images

def get_augmented_rotate(path):
    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE) # read image, image size is 180x180
    img=255-img
    img = cv2.resize(img, (RESIZE_DIM,RESIZE_DIM) ,interpolation=cv2.INTER_AREA)
  
 
    M = cv2.getRotationMatrix2D((RESIZE_DIM/2,RESIZE_DIM/2),np.random.randint(1,90),1)
    dst = cv2.warpAffine(img,M,(RESIZE_DIM,RESIZE_DIM))
    
    gaussian_3 = cv2.GaussianBlur(dst, (9,9), 10.0) #unblur
    img = cv2.addWeighted(dst, 1.5, gaussian_3, -0.5, 0, dst)
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
    img = cv2.filter2D(img, -1, kernel)
    ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)  
    
    
    return dst

#Samplewise_centers
def get_augmented_blur(path):
    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE) # read image, image size is 180x180
    img=255-img
    img = cv2.resize(img, (RESIZE_DIM,RESIZE_DIM) ,interpolation=cv2.INTER_AREA)
    
    
    kernel = np.ones((9,9),np.float32)/80
    dst = cv2.filter2D(img,-1,kernel)
    
    gaussian_3 = cv2.GaussianBlur(dst, (9,9), 10.0) #unblur
    img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
    img = cv2.filter2D(img, -1, kernel)
    ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)  
    return dst


def get_augmented_zca_whitening(train_data,val_data):
    datagen = ImageDataGenerator(zca_whitening=True)
    train_data = train_data.reshape(-1, RESIZE_DIM,RESIZE_DIM,1)
    datagen.fit(train_data)
    for X_batch in datagen.flow(train_data, batch_size=len(train_data)):
        x=len(train_data)
        zca_img =[]
        for i in range(0, x):
            X_batch=X_batch.reshape(-1,RESIZE_DIM,RESIZE_DIM)
            X_batch[i]= X_batch[i].reshape(RESIZE_DIM,RESIZE_DIM)
            zca_img.append(X_batch[i])
        break
    zca_img = np.asarray(zca_img)
    return zca_img,val_data


def get_augmented_shear_range(path):
    datagen = ImageDataGenerator(shear_range = 50)
    # fit parameters from data
    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE) # read image, image size is 180x180
    img=255-img
    img = cv2.resize(img, (RESIZE_DIM,RESIZE_DIM) ,interpolation=cv2.INTER_AREA)
    gaussian_3 = cv2.GaussianBlur(img, (9,9), 10.0) #unblur
    img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
    img = cv2.filter2D(img, -1, kernel)
    ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)  
    
    img = img.reshape(1, RESIZE_DIM, RESIZE_DIM, 1)
    # configure batch size and retrieve one batch of images
    for X_batch in datagen.flow(img):
    # create a grid of 3x3 images

        for i in range(0, 1):
            X_batch=X_batch.reshape(-1,RESIZE_DIM,RESIZE_DIM)
            
            X_batch[0]= X_batch[0].reshape(RESIZE_DIM,RESIZE_DIM)
        break
    return X_batch[0]

def get_augmented_wshift(path):
    datagen = ImageDataGenerator(width_shift_range = 25)
    # fit parameters from data
    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE) # read image, image size is 180x180
    img=255-img
    img = cv2.resize(img, (RESIZE_DIM,RESIZE_DIM) ,interpolation=cv2.INTER_AREA)
    gaussian_3 = cv2.GaussianBlur(img, (9,9), 10.0) #unblur
    img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
    img = cv2.filter2D(img, -1, kernel)
    ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)  
    
    img = img.reshape(1, RESIZE_DIM, RESIZE_DIM, 1)
    # configure batch size and retrieve one batch of images
    for X_batch in datagen.flow(img):
    # create a grid of 3x3 images

        for i in range(0, 1):
            X_batch=X_batch.reshape(-1,RESIZE_DIM,RESIZE_DIM)
            X_batch[0]= X_batch[0].reshape(RESIZE_DIM,RESIZE_DIM)
        break
    return X_batch[0]


def get_augmented_hshift(path):
    datagen = ImageDataGenerator(height_shift_range = 25)
    # fit parameters from data
    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE) # read image, image size is 180x180
    img=255-img
    img = cv2.resize(img, (RESIZE_DIM,RESIZE_DIM) ,interpolation=cv2.INTER_AREA)
    gaussian_3 = cv2.GaussianBlur(img, (9,9), 10.0) #unblur
    img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
    img = cv2.filter2D(img, -1, kernel)
    ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)  
    
    img = img.reshape(1, RESIZE_DIM, RESIZE_DIM, 1)
    # configure batch size and retrieve one batch of images
    for X_batch in datagen.flow(img):
    # create a grid of 3x3 images

        for i in range(0, 1):
            X_batch=X_batch.reshape(-1,28,28)
            X_batch[0]= X_batch[0].reshape(28,28)
        break
    return X_batch[0]


def get_augmented_w_h_shift(path):
    datagen = ImageDataGenerator(height_shift_range = 25,width_shift_range = 25)
    # fit parameters from data
    img=cv2.imread(path,cv2.IMREAD_GRAYSCALE) # read image, image size is 180x180
    img=255-img
    img = cv2.resize(img, (RESIZE_DIM,RESIZE_DIM) ,interpolation=cv2.INTER_AREA)
    gaussian_3 = cv2.GaussianBlur(img, (9,9), 10.0) #unblur
    img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
    img = cv2.filter2D(img, -1, kernel)
    ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)  
    
    img = img.reshape(1, RESIZE_DIM, RESIZE_DIM, 1)
    img = img.reshape(1, RESIZE_DIM, RESIZE_DIM, 1)
    # configure batch size and retrieve one batch of images
    for X_batch in datagen.flow(img):
    # create a grid of 3x3 images

        for i in range(0, 1):
            X_batch=X_batch.reshape(-1,28,28)
            X_batch[0]= X_batch[0].reshape(28,28)
        break
    return X_batch[0]



def get_data_aug(paths_img,path_label=None,resize_dim=None):
    '''reads images from the filepaths, resizes them (if given), and returns them in a numpy array
    Args:
        paths_img: image filepaths
        path_label: pass image label filepaths while processing training data, defaults to None while processing testing data
        resize_dim: if given, the image is resized to resize_dim x resize_dim (optional)
    Returns:
        X: group of images
        y: categorical true labels
    '''
    X=[] # initialize empty list for resized images
    for i,path in enumerate(paths_img): # details about enumerate is on : https://stackoverflow.com/questions/22171558/what-does-enumerate-mean
        #img=img_crop(path)
        img=get_augmented_rotate(path)
        X.append(img) # expand image to 28x28x1 and append to the list
        # display progress
        if i==len(paths_img)-1:
            end='\n'
        else:
            end='\r'
        print('processed {}/{}'.format(i+1,len(paths_img)),end=end)
        
    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename')  
        #by the upper line of code we are saying that we won't be using the conventional 0,1,2 indexing but we will be using the filenames as the index for each row data
        y_label=[df.loc[get_key(path)]['digit'] for path in  paths_img] # get the labels corresponding to the images
        #first the for loop creates path for all the the image file then only the image name is exracted from the path with the help of the function get_key
        #this filename is used as the index of the csv file and we are interested only the digit value of that image and therefore ['digit'] part
        y=to_categorical(y_label,10) # transfrom integer value to categorical variable
        return X, y
    
def get_data_aug2(paths_img,path_label=None,resize_dim=None):
    '''reads images from the filepaths, resizes them (if given), and returns them in a numpy array
    Args:
        paths_img: image filepaths
        path_label: pass image label filepaths while processing training data, defaults to None while processing testing data
        resize_dim: if given, the image is resized to resize_dim x resize_dim (optional)
    Returns:
        X: group of images
        y: categorical true labels
    '''
    X=[] # initialize empty list for resized images
    for i,path in enumerate(paths_img): # details about enumerate is on : https://stackoverflow.com/questions/22171558/what-does-enumerate-mean
        #img=img_crop(path)
        img=get_augmented_blur(path)
        X.append(img) # expand image to 28x28x1 and append to the list
        # display progress
        if i==len(paths_img)-1:
            end='\n'
        else: 
            end='\r'
        print('processed {}/{}'.format(i+1,len(paths_img)),end=end)
        
    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename')  
        #by the upper line of code we are saying that we won't be using the conventional 0,1,2 indexing but we will be using the filenames as the index for each row data
        y_label=[df.loc[get_key(path)]['digit'] for path in  paths_img] # get the labels corresponding to the images
        #first the for loop creates path for all the the image file then only the image name is exracted from the path with the help of the function get_key
        #this filename is used as the index of the csv file and we are interested only the digit value of that image and therefore ['digit'] part
        y=to_categorical(y_label,10) # transfrom integer value to categorical variable
        return X, y
    

def get_data_aug4(paths_img,path_label=None,resize_dim=None):
    '''reads images from the filepaths, resizes them (if given), and returns them in a numpy array
    Args:
        paths_img: image filepaths
        path_label: pass image label filepaths while processing training data, defaults to None while processing testing data
        resize_dim: if given, the image is resized to resize_dim x resize_dim (optional)
    Returns:
        X: group of images
        y: categorical true labels
    '''
    X=[] # initialize empty list for resized images
    for i,path in enumerate(paths_img): # details about enumerate is on : https://stackoverflow.com/questions/22171558/what-does-enumerate-mean
        #img=img_crop(path)
        img=get_augmented_shear_range(path)
        X.append(img) # expand image to 28x28x1 and append to the list
        # display progress
        if i==len(paths_img)-1:
            end='\n'
        else: 
            end='\r'
        print('processed {}/{}'.format(i+1,len(paths_img)),end=end)
        
    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename')  
        #by the upper line of code we are saying that we won't be using the conventional 0,1,2 indexing but we will be using the filenames as the index for each row data
        y_label=[df.loc[get_key(path)]['digit'] for path in  paths_img] # get the labels corresponding to the images
        #first the for loop creates path for all the the image file then only the image name is exracted from the path with the help of the function get_key
        #this filename is used as the index of the csv file and we are interested only the digit value of that image and therefore ['digit'] part
        y=to_categorical(y_label,10) # transfrom integer value to categorical variable
        return X, y

def get_data_aug5(paths_img,path_label=None,resize_dim=None):
    '''reads images from the filepaths, resizes them (if given), and returns them in a numpy array
    Args:
        paths_img: image filepaths
        path_label: pass image label filepaths while processing training data, defaults to None while processing testing data
        resize_dim: if given, the image is resized to resize_dim x resize_dim (optional)
    Returns:
        X: group of images
        y: categorical true labels
    '''
    X=[] # initialize empty list for resized images
    for i,path in enumerate(paths_img): # details about enumerate is on : https://stackoverflow.com/questions/22171558/what-does-enumerate-mean
        #img=img_crop(path)
        img=get_augmented_wshift(path)
        X.append(img) # expand image to 28x28x1 and append to the list
        # display progress
        if i==len(paths_img)-1:
            end='\n'
        else: 
            end='\r'
        print('processed {}/{}'.format(i+1,len(paths_img)),end=end)
        
    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename')  
        #by the upper line of code we are saying that we won't be using the conventional 0,1,2 indexing but we will be using the filenames as the index for each row data
        y_label=[df.loc[get_key(path)]['digit'] for path in  paths_img] # get the labels corresponding to the images
        #first the for loop creates path for all the the image file then only the image name is exracted from the path with the help of the function get_key
        #this filename is used as the index of the csv file and we are interested only the digit value of that image and therefore ['digit'] part
        y=to_categorical(y_label,10) # transfrom integer value to categorical variable
        return X, y
    
def get_data_aug6(paths_img,path_label=None,resize_dim=None):
    '''reads images from the filepaths, resizes them (if given), and returns them in a numpy array
    Args:
        paths_img: image filepaths
        path_label: pass image label filepaths while processing training data, defaults to None while processing testing data
        resize_dim: if given, the image is resized to resize_dim x resize_dim (optional)
    Returns:
        X: group of images
        y: categorical true labels
    '''
    X=[] # initialize empty list for resized images
    for i,path in enumerate(paths_img): # details about enumerate is on : https://stackoverflow.com/questions/22171558/what-does-enumerate-mean
        #img=img_crop(path)
        img=get_augmented_hshift(path)
        X.append(img) # expand image to 28x28x1 and append to the list
        # display progress
        if i==len(paths_img)-1:
            end='\n'
        else: 
            end='\r'
        print('processed {}/{}'.format(i+1,len(paths_img)),end=end)
        
    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename')  
        #by the upper line of code we are saying that we won't be using the conventional 0,1,2 indexing but we will be using the filenames as the index for each row data
        y_label=[df.loc[get_key(path)]['digit'] for path in  paths_img] # get the labels corresponding to the images
        #first the for loop creates path for all the the image file then only the image name is exracted from the path with the help of the function get_key
        #this filename is used as the index of the csv file and we are interested only the digit value of that image and therefore ['digit'] part
        y=to_categorical(y_label,10) # transfrom integer value to categorical variable
        return X, y

def get_data_aug7(paths_img,path_label=None,resize_dim=None):
    '''reads images from the filepaths, resizes them (if given), and returns them in a numpy array
    Args:
        paths_img: image filepaths
        path_label: pass image label filepaths while processing training data, defaults to None while processing testing data
        resize_dim: if given, the image is resized to resize_dim x resize_dim (optional)
    Returns:
        X: group of images
        y: categorical true labels
    '''
    X=[] # initialize empty list for resized images
    for i,path in enumerate(paths_img): # details about enumerate is on : https://stackoverflow.com/questions/22171558/what-does-enumerate-mean
        #img=img_crop(path)
        img=get_augmented_w_h_shift(path)
        X.append(img) # expand image to 28x28x1 and append to the list
        # display progress
        if i==len(paths_img)-1:
            end='\n'
        else: end='\r'
        print('processed {}/{}'.format(i+1,len(paths_img)),end=end)
        
    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename')  
        #by the upper line of code we are saying that we won't be using the conventional 0,1,2 indexing but we will be using the filenames as the index for each row data
        y_label=[df.loc[get_key(path)]['digit'] for path in  paths_img] # get the labels corresponding to the images
        #first the for loop creates path for all the the image file then only the image name is exracted from the path with the help of the function get_key
        #this filename is used as the index of the csv file and we are interested only the digit value of that image and therefore ['digit'] part
        y=to_categorical(y_label,10) # transfrom integer value to categorical variable
        return X, y


def get_data(paths_img,path_label=None,resize_dim=None):
    '''reads images from the filepaths, resizes them (if given), and returns them in a numpy array
    Args:
        paths_img: image filepaths
        path_label: pass image label filepaths while processing training data, defaults to None while processing testing data
        resize_dim: if given, the image is resized to resize_dim x resize_dim (optional)
    Returns:
        X: group of images
        y: categorical true labels
    '''
    X=[] # initialize empty list for resized images
    
    for i,path in enumerate(paths_img): # details about enumerate is on : https://stackoverflow.com/questions/22171558/what-does-enumerate-mean
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=255-img
        if resize_dim is not None:
            try:
                img=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA) # resize image to 28x28
#            X.append(np.expand_dims(img,axis=2)) # expand image to 28x28x1 and append to the list.
                
            except:
                print('The image at %s could not be resized after cropping\n so we are using the previous method'  %path)
                img=cv2.imread(path,cv2.IMREAD_GRAYSCALE) # read image, image size is 180x180
                img=255-img
                img=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA)
        #Attempt at using a Gaussian filter here:
        
        gaussian_3 = cv2.GaussianBlur(img, (9,9), 10.0) #unblur
        img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
        img = cv2.filter2D(img, -1, kernel)
        #thresh = 200
        #maxValue = 255
        #th, img = cv2.threshold(img, thresh, maxValue, cv2.THRESH_BINARY);
        ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)        
        
        
        X.append(img) # expand image to 28x28x1 and append to the list
        # display progress
        if i==len(paths_img)-1:
            end='\n'
        else: end='\r'
        print('processed {}/{}'.format(i+1,len(paths_img)),end=end)
        
    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename')  
        #by the upper line of code we are saying that we won't be using the conventional 0,1,2 indexing but we will be using the filenames as the index for each row data
        y_label=[df.loc[get_key(path)]['digit'] for path in  paths_img] # get the labels corresponding to the images
        #first the for loop creates path for all the the image file then only the image name is exracted from the path with the help of the function get_key
        #this filename is used as the index of the csv file and we are interested only the digit value of that image and therefore ['digit'] part
        y=to_categorical(y_label,10) # transfrom integer value to categorical variable
        return X, y

def imshow_group(X,y=None,y_pred=None,n_per_row=10):
    '''helper function to visualize a group of images along with their categorical true labels (y) and prediction probabilities.
    Args:
        X: images
        y: categorical true labels
        y_pred: predicted class probabilities
        n_per_row: number of images per row to be plotted
    '''
    n_sample=len(X)
    img_dim=X.shape[1]
    
    j=np.ceil(n_sample/n_per_row)
    
    fig=plt.figure(figsize=(FIG_WIDTH,ROW_HEIGHT*j))
    
    for i,img in enumerate(X):
        plt.subplot(j,n_per_row,i+1)
        plt.imshow(img, cmap='gray')
        if y is not None:
                plt.title('true label: {}'.format(np.argmax(y[i])))
                #argmax function finds the maximum elemant value in an array and returns that elements index number
        if y_pred is not None:
            top_n=3 # top 3 predictions with highest probabilities
            
            ind_sorted=np.argsort(y_pred[i])[::-1] 
            #^ this line of code basically sort the array in an ascending order by their index value
            #for example if you give an array like this : [-9,-4,-1,0,1,8,6,2,-10,0] the fuction will return an array like this : [8, 0, 1, 2, 3, 9, 4, 7, 6, 5]
            #the -1 part select the last element
            h=img_dim+4
            
            for k in range(top_n):
                string='pred: {} ({:.0f}%)\n'.format(ind_sorted[k],y_pred[i,ind_sorted[k]]*100)
                plt.text(img_dim/2, h, string, horizontalalignment='center',verticalalignment='center')
                h+=4
        plt.axis('off')
    plt.show()   #shows the plot
    
    
    
def create_submission(predictions,keys,path):
    result = pd.DataFrame(
        predictions,
        columns=['label'],
        index=keys
        )
    result.index.name='key'
    result.to_csv(path, index=True)

# Preprocess data using utility functions and augment each data in 7 ways

In [ ]:
X_train_a,y_train_a=get_data(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)
X_train_b,y_train_b=get_data(paths_train_b,path_label_train_b,resize_dim=RESIZE_DIM)
X_train_c,y_train_c=get_data(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)
X_train_d,y_train_d=get_data(paths_train_d,path_label_train_d,resize_dim=RESIZE_DIM)
X_train_e,y_train_e=get_data(paths_train_e,path_label_train_e,resize_dim=RESIZE_DIM)


#Extra dataset after applying various transformations on the image
X_train_aug_rotate, y_train_aug_rotate = get_data_aug(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)
X_train_aug_shear, y_train_aug_shear = get_data_aug4(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)
X_train_aug_wshift, y_train_aug_wshift = get_data_aug5(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)
X_train_aug_hshift, y_train_aug_hshift = get_data_aug6(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)
X_train_w_h_shift, y_train_w_h_shift = get_data_aug7(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)
X_train_aug_blur, y_train_aug_blur = get_data_aug2(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)

#zca
X_train_aug_zca, y_train_aug_zca = get_augmented_zca_whitening(X_train_a,y_train_a)



#Now add train b to the following as well


X_train_aug_rotate2, y_train_aug_rotate2 = get_data_aug(paths_train_b,path_label_train_b,resize_dim=RESIZE_DIM)
X_train_aug_shear2, y_train_aug_shear2 = get_data_aug4(paths_train_b,path_label_train_b,resize_dim=RESIZE_DIM)
X_train_aug_wshift2, y_train_aug_wshift2 = get_data_aug5(paths_train_b,path_label_train_b,resize_dim=RESIZE_DIM)
X_train_aug_hshift2, y_train_aug_hshift2 = get_data_aug6(paths_train_b,path_label_train_b,resize_dim=RESIZE_DIM)
X_train_w_h_shift2, y_train_w_h_shift2 = get_data_aug7(paths_train_b,path_label_train_b,resize_dim=RESIZE_DIM)
X_train_aug_blur2, y_train_aug_blur2 = get_data_aug2(paths_train_b,path_label_train_b,resize_dim=RESIZE_DIM)

#zca
X_train_aug_zca2, y_train_aug_zca2 = get_augmented_zca_whitening(X_train_b,y_train_b)

X_train_aug_rotate3, y_train_aug_rotate3 = get_data_aug(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)
X_train_aug_shear3, y_train_aug_shear3 = get_data_aug4(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)
X_train_aug_wshift3, y_train_aug_wshift3 = get_data_aug5(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)
X_train_aug_hshift3, y_train_aug_hshift3 = get_data_aug6(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)
X_train_w_h_shift3, y_train_w_h_shift3 = get_data_aug7(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)
X_train_aug_blur3, y_train_aug_blur3 = get_data_aug2(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)

#zca
X_train_aug_zca3, y_train_aug_zca3 = get_augmented_zca_whitening(X_train_c,y_train_c)


X_train_aug_rotate4, y_train_aug_rotate4 = get_data_aug(paths_train_d,path_label_train_d,resize_dim=RESIZE_DIM)
X_train_aug_shear4, y_train_aug_shear4 = get_data_aug4(paths_train_d,path_label_train_d,resize_dim=RESIZE_DIM)
X_train_aug_wshift4, y_train_aug_wshift4 = get_data_aug5(paths_train_d,path_label_train_d,resize_dim=RESIZE_DIM)
X_train_aug_hshift4, y_train_aug_hshift4 = get_data_aug6(paths_train_d,path_label_train_d,resize_dim=RESIZE_DIM)
X_train_w_h_shift4, y_train_w_h_shift4 = get_data_aug7(paths_train_d,path_label_train_d,resize_dim=RESIZE_DIM)
X_train_aug_blur4, y_train_aug_blur4 = get_data_aug2(paths_train_d,path_label_train_d,resize_dim=RESIZE_DIM)

#zca
X_train_aug_zca4, y_train_aug_zca4 = get_augmented_zca_whitening(X_train_d,y_train_d)


X_train_aug_rotate5, y_train_aug_rotate5 = get_data_aug(paths_train_e,path_label_train_e,resize_dim=RESIZE_DIM)
X_train_aug_shear5, y_train_aug_shear5 = get_data_aug4(paths_train_e,path_label_train_e,resize_dim=RESIZE_DIM)
X_train_aug_wshift5, y_train_aug_wshift5 = get_data_aug5(paths_train_e,path_label_train_e,resize_dim=RESIZE_DIM)
X_train_aug_hshift5, y_train_aug_hshift5 = get_data_aug6(paths_train_e,path_label_train_e,resize_dim=RESIZE_DIM)
X_train_w_h_shift5, y_train_w_h_shift5 = get_data_aug7(paths_train_e,path_label_train_e,resize_dim=RESIZE_DIM)
X_train_aug_blur5, y_train_aug_blur5 = get_data_aug2(paths_train_e,path_label_train_e,resize_dim=RESIZE_DIM)

#zca
X_train_aug_zca5, y_train_aug_zca5 = get_augmented_zca_whitening(X_train_e,y_train_e)





# Concatenate all data in one variable

In [ ]:
X_train_all=np.concatenate((X_train_a,X_train_b,X_train_c,X_train_d,X_train_e,X_train_aug_rotate, X_train_aug_shear, X_train_aug_wshift, X_train_aug_hshift,X_train_w_h_shift,X_train_aug_blur, X_train_aug_zca, X_train_aug_rotate2, X_train_aug_shear2, X_train_aug_wshift2, X_train_aug_hshift2,X_train_w_h_shift2,X_train_aug_blur2, X_train_aug_zca2,X_train_aug_rotate3, X_train_aug_shear3, X_train_aug_wshift3, X_train_aug_hshift3,X_train_w_h_shift3,X_train_aug_blur3, X_train_aug_zca3,X_train_aug_rotate4, X_train_aug_shear4, X_train_aug_wshift4, X_train_aug_hshift4,X_train_w_h_shift4,X_train_aug_blur4, X_train_aug_zca4,X_train_aug_rotate5, X_train_aug_shear5, X_train_aug_wshift5, X_train_aug_hshift5,X_train_w_h_shift5,X_train_aug_blur5, X_train_aug_zca5),axis=0)

y_train_all=np.concatenate((y_train_a,y_train_b,y_train_c,y_train_d,y_train_e,y_train_aug_rotate, y_train_aug_shear, y_train_aug_wshift, y_train_aug_hshift, y_train_w_h_shift,y_train_aug_blur, y_train_aug_zca,y_train_aug_rotate2, y_train_aug_shear2, y_train_aug_wshift2, y_train_aug_hshift2, y_train_w_h_shift2,y_train_aug_blur2, y_train_aug_zca2,y_train_aug_rotate3, y_train_aug_shear3, y_train_aug_wshift3, y_train_aug_hshift3, y_train_w_h_shift3,y_train_aug_blur3, y_train_aug_zca3,y_train_aug_rotate4, y_train_aug_shear4, y_train_aug_wshift4, y_train_aug_hshift4, y_train_w_h_shift4,y_train_aug_blur4, y_train_aug_zca4,y_train_aug_rotate5, y_train_aug_shear5, y_train_aug_wshift5, y_train_aug_hshift5, y_train_w_h_shift5,y_train_aug_blur5, y_train_aug_zca5),axis=0)

X_train_all = np.reshape(X_train_all,(-1,RESIZE_DIM,RESIZE_DIM, 1 ))
X_train_all.shape , y_train_all.shape

In [ ]:
X_test_a=get_data(paths_test_a,resize_dim=RESIZE_DIM)
X_test_b=get_data(paths_test_b,resize_dim=RESIZE_DIM)
X_test_c=get_data(paths_test_c,resize_dim=RESIZE_DIM)
X_test_d=get_data(paths_test_d,resize_dim=RESIZE_DIM)
X_test_e=get_data(paths_test_e,resize_dim=RESIZE_DIM)
X_test_f=get_data(paths_test_f,resize_dim=RESIZE_DIM)
X_test_auga=get_data(paths_test_auga,resize_dim=RESIZE_DIM)
X_test_augc=get_data(paths_test_augc,resize_dim=RESIZE_DIM)

# Getting the test data (But we do not have the validation for test data)

In [ ]:
X_test_all=np.concatenate((X_test_a,X_test_b,X_test_c,X_test_d,X_test_e,X_test_f,X_test_auga,X_test_augc))
X_test_all = np.reshape(X_test_all,((-1,RESIZE_DIM,RESIZE_DIM, 1 )))
X_test_all.shape

# Making test-train split from our test data

### Since we don't have the validation for test data so we are spliting the train data in 80% and 20%

In [ ]:
indices=list(range(len(X_train_all)))
np.random.seed(42)
np.random.shuffle(indices)

ind=int(len(indices)*0.80)
# train data
X_train=X_train_all[indices[:ind]] 
y_train=y_train_all[indices[:ind]]

# validation data
X_val=X_train_all[indices[-(len(indices)-ind):]] #-ve indexing means start from the last 
# -1 means the last element similarly -2 means the 2nd last element and -n mens the nth ast element

y_val=y_train_all[indices[-(len(indices)-ind):]]


# Model

In [ ]:
def get_model():
    input_layer=Input(shape=(RESIZE_DIM,RESIZE_DIM,1))
    x=Conv2D(filters=32,kernel_size=(3,3),padding='same', activation='relu')(input_layer)
    x=Conv2D(filters=32,kernel_size=(3,3),padding='same', activation='relu')(x)
    x=MaxPool2D(pool_size=(2,2),strides=2,padding='valid')(x)
    
    
    x=Conv2D(filters=64,kernel_size=(3,3),padding='same', activation='relu')(input_layer)
    x=Conv2D(filters=64,kernel_size=(3,3),padding='same', activation='relu')(x)
    x=MaxPool2D(pool_size=(2,2),strides=2,padding='valid')(x)
    
    x=Conv2D(filters=128,kernel_size=(3,3),padding='same', activation='relu')(x)
    x=Conv2D(filters=128,kernel_size=(3,3),padding='same', activation='relu')(x)
    x=MaxPool2D(pool_size=(2,2),strides=2,padding='valid')(x)
    
    
    x=Conv2D(filters=256,kernel_size=(3,3),padding='same', activation='relu')(x)
    x=Conv2D(filters=256,kernel_size=(3,3),padding='same', activation='relu')(x)
    x=MaxPool2D(pool_size=(2,2),strides=2,padding='valid')(x)
    
    x=Conv2D(filters=512,kernel_size=(3,3),padding='same', activation='relu')(x)
    x=Conv2D(filters=512,kernel_size=(3,3),padding='same', activation='relu')(x)
    x=MaxPool2D(pool_size=(2,2),strides=2,padding='valid')(x)
    
    x=Flatten()(x)
    x=Dense(units=512, activation='relu', W_constraint=maxnorm(3))(x)
    x=Dense(units=1024, activation='relu', W_constraint=maxnorm(3))(x)
    x=Dense(units=256, activation='relu', W_constraint=maxnorm(3))(x)
    x=Dense(units=10)(x)    
    output_layer=Activation('softmax')(x)
    model=Model(inputs=input_layer,outputs=output_layer)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
    return model



model=get_model()
model.summary()

# Train and Validate

In [ ]:
weight_file='best.hdf5' # save best model at this location after each epoch
check  = ModelCheckpoint(weight_file, monitor = 'val_acc', verbose=1, save_best_only=True, mode= max )
checkpoints = [check]
K.tensorflow_backend.clear_session() # destroys the current graph and builds a new one
model=get_model() # create the model
K.set_value(model.optimizer.lr,1e-4) # set the learning rate
# fit the model

#model.load_weights('best.hdf5')
h=model.fit(x=X_train,     
            y=y_train, 
            batch_size=64, 
            epochs=20, 
            verbose=1, 
            validation_data=(X_val,y_val),
            shuffle=True,
            callbacks = checkpoints
            )

# Make Predictions on Test Set

In [ ]:
model.load_weights(weight_file)
predictions_prob=model.predict(X_test_all) # get predictions for all the testing data

### Let's observe a few predictions.

In [ ]:
n_sample=200
np.random.seed(42)
ind=np.random.randint(0,len(X_test_all), size=n_sample)

In [ ]:
X_test_all = np.reshape(X_test_all,(-1,RESIZE_DIM,RESIZE_DIM)) #reshaping to plot the images
imshow_group(X=X_test_all[ind],y=None,y_pred=predictions_prob[ind])

# Create Submission File

In [ ]:
labels=[np.argmax(pred) for pred in predictions_prob]

In [ ]:
keys=[get_key(path) for path in paths_test_all ]

In [ ]:
create_submission(predictions=labels,keys=keys,path='file2.csv')